In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'exercises:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5016496%2F8425007%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240523%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240523T182803Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D986503aeb89e71b03e3e70417c9eda798d95b7e6abf5f36a2c53e571fc3b8c4c6fb5a2d0a57045578737c17f6efaca02cd82b78bb4799ab84661a20e58c903a38de5f36f06d5e0b9ddce985eb24f4c14111451c0012664c35241f91e2ed656ebd6a71ab6c2a5d00e0626af2087073e0ca807d9182c24580f798504dc7208d64a506779e836fdf4e2526f98a27ce6f322126c9b9de5922fc979a8abb1898b5a1dd3e5a03a11bd44c7c642213a4b4b0d17addd52e29faa669dda3d8b1450932522faad2698180c5de04d05ddb12091f3d8d04b3b50734bf58fc61154b32c1c311a3b3607653c116cd22bb3f1c9d8b0fdb9248da23a80c96158aa5e6fb97dd7b078,subset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5057504%2F8479644%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240523%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240523T182804Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D14cb043f14c335698d67bbda78f82270a8273ef5defd61897058d793d7e15925a6a1e55c766ec2bf9ef1893181e8b5ca0abb52206624945ea6724b1632caee1870c4f503cddc3f4153489a4369f4be9ad5e5e5111cc01fc49fcfebf2e8225af8b22b94261770ee5a464195b1dc504c08367f3e7b8742d77018bd91127cc52ecea19bc37239f20de622022e62486e82dce8d74810daa276b5229ae3078c17e5adaa38bbabb5f1533bb439290c7656b3235d6d615f7aa86e788bdca57397389c342ffab75b705dab46915dfd0bdb3e708a9c5fcc3173b4d0e104ba8b407e7eaa5022c0747298cc1de67921b6cfd93a28242bc16106a5510d641144a97e05bcb988'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 30282 bytes downloaded
Downloaded and uncompressed: exercises
[==================================================] 5372 bytes downloaded
Downloaded and uncompressed: subset
Data source import complete.


In [2]:
!pip install peft
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 39.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [3]:
def reward_model(code, unit_tests):
    '''
    This function should take in a code snippet and a list of unit tests.
    It should return a score that represents how well the code snippet.
    The score is yet to be defined.
    I assume the score will depend on 2 things:
    1. Are there any syntax errors in the code snippet ?
    2. Does the code snippet pass all the unit tests ?
    The score should take into account both of these factors.
    '''
    # If the code is empty, return a specific score
    if code == '':
        print("Code snippet is empty, parsing failed")
        return -2

    # Check for syntax errors
    try:
        compile(code, '<string>', 'exec')
    except SyntaxError as e:
        print(f"Syntax Error: {e}")
        return -1.0

    # Combine the code and unit tests into a single script
    script = code + '\n' + unit_tests
    namespace = {}

    # Check for NameError and other runtime errors
    try:
        exec(script, namespace)
    except NameError as e:
        print(f"Name Error: {e}")
        return -0.6
    except RuntimeError as e:
        print(f"Runtime Error: {e}")
        return -0.6
    except AssertionError as e:
        print(f"Assertion Error: {e}")
        return -0.3
    except Exception as e:
        print(f"Other Error: {e}")
        return -0.4

    # If no exceptions, return a positive score
    print("Code snippet passed all unit tests")
    return 1.0

In [4]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
from datasets import load_dataset
from tqdm import tqdm

In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# We setup the LoRA configuration

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# We import model and tokenizer

checkpoint = "microsoft/phi-2"
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    checkpoint, device_map={"": device}, peft_config=peft_config
)
tokenizer = AutoTokenizer.from_pretrained(
    checkpoint, padding_side="left", pad_token='<pad>'
)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token

print_trainable_parameters(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


trainable params: 9177601 || all params: 2788861441 || trainable%: 0.3290805654621979


In [7]:
data_path = "/input/subset/exercises_sub_dataset.json"

ppo_config = PPOConfig(
    model_name="microsoft/phi-2",
    learning_rate=1e-5,
    remove_unused_columns=False,
    batch_size=8,
    mini_batch_size=2
    # More hyperparameters can be added here
)

def preprocess_function(examples):
    input_ids = []
    attention_masks = []
    unit_tests = []
    prefix = '''Instruct: For the following function please only generate the solution and only the solution.
    After you finish generating the function asked you should stop the generation. The python code starts with a ```python
 and once you finish generating the function you should close the code with a ```.\nOutput:\n```python\n
    '''

    for docstring, unit_test in zip(examples['docstring'], examples['unit_tests']):
        prompt = prefix + '\n' + docstring
        tokenized = tokenizer(prompt, truncation=True)
        input_ids.append(torch.tensor(tokenized['input_ids']))
        unit_tests.append(unit_test)

    return {
        "input_ids": input_ids,
        "unit_tests": unit_tests
    }

def data_collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

dataset = load_dataset('json', data_files=data_path, split='train')

ds = dataset.map(
    preprocess_function,
    batched=True
)

ds = ds.filter(lambda x: len(x["input_ids"]) < 512, batched=False)

ds.set_format(type="torch")

trainer = PPOTrainer(
    model=model,
    tokenizer=tokenizer,
    config=ppo_config,
    dataset=ds,
    data_collator=data_collator
)

generation_kwargs = {
    # "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": 100_000,
    "max_new_tokens": 200,
    "early_stopping": True,
}

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Filter:   0%|          | 0/23 [00:00<?, ? examples/s]

In [8]:
ds

Dataset({
    features: ['entry_point', 'title', 'unit_tests', 'description', 'docstring', 'input_ids'],
    num_rows: 23
})

In [9]:
import re

def format_response(response):
    lines = response.split('\n')
    lines = lines[:-1]
    modified_solution = '\n'.join(lines)
    print('####### [RESPONSE] ########')
    print(response)
    print('####### [RESPONSE\] ########')
    return modified_solution

def print_prompt(query_tensor):
    prompt = tokenizer.decode(query_tensor.squeeze())
    print('####### PROMPT ########')
    print(prompt)
    print('####### PROMPT\ ########')


def extract_code(text, occurrence=1):
    # Using regular expression to find all occurrences of text enclosed in triple backticks
    codes = re.findall(r'```python(.*?)```', text, re.DOTALL)
    if len(codes) > occurrence:
        return codes[occurrence].strip()
    else:
        print('We could not parse the response')
        return ''

In [10]:
epochs = 10
for epoch in tqdm(range(epochs), "epoch: "):
    for batch in tqdm(trainer.dataloader, "batch: "):
        query_tensors = batch['input_ids']

        response_tensors = trainer.generate(query_tensors, **generation_kwargs)
        batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)
        batch["response"] = [extract_code(r) for r in batch["response"]]

        texts = [r for r in batch['response']]
        unit_tests = [tests for tests in batch['unit_tests']]
        rewards = [torch.tensor(reward_model(code, tests)) for code, tests in zip(texts, unit_tests)]
        print(rewards)

        stats = trainer.step(query_tensors, response_tensors, rewards)
        trainer.log_stats(stats, batch, rewards)

batch:   0%|          | 0/2 [00:00<?, ?it/s]You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


We could not parse the response
We could not parse the response
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Code snippet is empty, parsing failed
Code snippet is empty, parsing failed
Syntax Error: 'return' outside function (<string>, line 19)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Name Error: name 'List' is not defined
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
[tensor(-1.), tensor(-2), tensor(-2), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-0.6000), tensor(-1.)]



batch:  50%|█████     | 1/2 [01:02<01:02, 62.06s/it]

We could not parse the response
We could not parse the response
We could not parse the response
Code snippet is empty, parsing failed
Name Error: name 'List' is not defined
Code snippet is empty, parsing failed
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Code snippet is empty, parsing failed
[tensor(-2), tensor(-0.6000), tensor(-2), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-2)]



batch:   0%|          | 0/2 [00:00<?, ?it/s]

...


batch:   0%|          | 0/2 [00:00<?, ?it/s]

We could not parse the response
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Code snippet is empty, parsing failed
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: unmatched ')' (<string>, line 12)
Syntax Error: unexpected indent (<string>, line 12)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
[tensor(-1.), tensor(-2), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-1.)]


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:1289: UserWarning: KL divergence is starting to become negative: -1.46 - this might be a precursor for failed training. sometimes this happens because the generation kwargs are not correctly set. Please make sure that the generation kwargs are set correctly, or review your training hyperparameters.
  warnings.warn(

batch:  50%|█████     | 1/2 [01:02<01:02, 62.95s/it]

We could not parse the response
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: unexpected indent (<string>, line 13)
Code snippet is empty, parsing failed
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
Syntax Error: unterminated triple-quoted string literal (detected at line 14) (<string>, line 13)
Syntax Error: expected an indented block after function definition on line 1 (<string>, line 2)
[tensor(-1.), tensor(-1.), tensor(-2), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-1.), tensor(-1.)]



epoch: 100%|██████████| 10/10 [21:15<00:00, 127.51s/it]
